In [69]:
import pandas as pd
import doctest
df = pd.read_csv('data_for_futures.csv')
df.head(4)
#df.columns

,Unnamed: 0,date,sp_price,sp_dividend,sp_real_price,sp_real_dividend
0,7,1871.01,4.44,0.26,114.541036,6.707358
1,8,1871.02,4.5,0.26,112.649234,6.508622
2,9,1871.03,4.61,0.26,113.717822,6.413587
3,10,1871.04,4.74,0.26,121.353738,6.656534


In [113]:
#define all functions
def simulate_portfolio(df, leverage=1, dividend=False, rebalance_period=1):
    """
    Simulate portfolio value given an S&P real-price column.

    Examples
    --------
    Basic two-row sanity check (10 % price rise → 10 % portfolio rise)

    >>> import pandas as pd, math
    >>> df = pd.DataFrame({'sp_real_price': [100, 110]})
    >>> test_leverage = 1
    >>> out = simulate_portfolio(df, leverage=test_leverage)
    >>> out[f'portfolio_{test_leverage}x'].tolist()
    [1.0, 1.1]

    """
    df = df.copy()

    df[f'portfolio_{leverage}x'] = 1.0
    portfolio_idx = df.columns.get_loc(f'portfolio_{leverage}x')

    last_rebalance = 0
    for i in range(1, len(df)):
        if i - last_rebalance == rebalance_period:
            if dividend:
                raise NotImplementedError("Dividend handling not built yet")

            base = df.iloc[last_rebalance, df.columns.get_loc('sp_real_price')]
            curr = df.iloc[i,            df.columns.get_loc('sp_real_price')]

            df.iat[i, portfolio_idx] = (
                df.iat[last_rebalance, portfolio_idx] *
                (curr / base) ** leverage
            )
            last_rebalance = i
        else:  # carry forward
            df.iat[i, portfolio_idx] = df.iat[i-1, portfolio_idx]

    return df

doctest.run_docstring_examples(simulate_portfolio, globals())


In [119]:

def identify_windows(df, window_size):
    """
    given a df, returns a list of lists of all possible sliding windows start/endpoints of a given window size. 
    
    example
    ----
    >>> import pandas as pd 
    >>> df = pd.DataFrame({
    ...  'date': [1,2,3],
    ...    'portfolio1':[100,200,400],
    ...    'portfolio2':[100,50,25]
    ...    })
    >>> out = identify_windows(df,1)
    >>> out 
    [[0, 1], [1, 2]]
    """
    windows = []
    for start in range(len(df) - window_size):
        end = start + window_size 
        windows.append([start,end])
    return windows


doctest.run_docstring_examples(identify_windows, globals())


In [123]:

def calc_window_returns(df, window_size, date_column, portfolio_columns = [] ):
    """
    calculates portfolio return over all possible rolling windows of a given size

    ----
    example:
    >>> import pandas as pd
    >>> df = pd.DataFrame({
    ...    'date': ['day1','day2','day3'],
    ...    'portfolio1':[100,200,2000],
    ...    'portfolio2':[100,50,5]
    ...    })
    >>> out = calc_window_returns(df,window_size=1, date_column = 'date', portfolio_columns=['portfolio1','portfolio2'])
    >>> out.equals(pd.DataFrame({
    ... 'window_dates':[['day1','day2'],['day2','day3']],
    ... 'portfolio1_returns':[2.0,10.0],
    ... 'portfolio2_returns':[0.5,0.1]
    ...  })) 
    True
    """
    df = df.copy()
    windows = identify_windows(df, window_size=window_size)
    window_dates = []
    for date_idx, window in enumerate(windows):
        start_idx = window[0]
        end_idx = window[1]
        window_dates.append([df.iloc[start_idx, df.columns.get_loc(date_column)],
                             df.iloc[end_idx, df.columns.get_loc(date_column)]])
    
    out = pd.DataFrame({
        'window_dates': window_dates,
        })

    #for each portfolio column, calculate the returns over the defined windows and add to the outdf
    for portfolio in portfolio_columns:
        #calculate the return for each window
        portfolio_returns = [] #captures return over the many possible windows
        for window in windows:
            start_idx = window[0]
            end_idx = window[1]
            #return is portfolio at end / portfolio at start
            window_return = df.iloc[end_idx, df.columns.get_loc(portfolio)] / df.iloc[start_idx, df.columns.get_loc(portfolio)]
            portfolio_returns.append(window_return)
        #add the returns to the out
        out[f'{portfolio}_returns'] = portfolio_returns
    #print(out)
    return out




doctest.run_docstring_examples(calc_window_returns, globals())



In [67]:
def boxplot_returns(returns_df, portfolio_columns):
    """
    creates boxplot of portfolio returns for as many portfolios as you defined
    """
    return returns_df[portfolio_columns].boxplot()



In [ ]:
def full_run():
    df = pd.DataFrame({
      'date': [1,2,3],
        'portfolio1':[100,200,400],
        'portfolio2':[100,50,25]
        })

In [73]:
#run the analysis

#sloppy portfolio make
add_1x = simulate_portfolio(df)
add_125x = simulate_portfolio(add_1x, leverage=1.25)
add_150x = simulate_portfolio(add_125x, leverage=1.50)
add_175x = simulate_portfolio(add_150x, leverage=1.75)
add_200x = simulate_portfolio(add_150x, leverage=2)
add_200x.columns



Index(['Unnamed: 0', 'date', 'sp_price', 'sp_dividend', 'sp_real_price',
       'sp_real_dividend', 'portfolio_1x', 'portfolio_1.25x', 'portfolio_1.5x',
       'portfolio_2x'],
      dtype='object')

In [101]:
returns = calc_window_returns(add_200x, window_size=360,date_column='date', portfolio_columns=['portfolio_1x', 'portfolio_1.25x', 'portfolio_1.5x', 'portfolio_2x'])
returns.iloc[-5:,:]

,window_dates,portfolio_1x_returns,portfolio_1.25x_returns,portfolio_1.5x_returns,portfolio_2x_returns
1490,"[1995.03, 2025.03]",5.456605,8.339753,12.746290,29.774540
1491,"[1995.04, 2025.04]",5.005839,7.487661,11.199932,25.058429
1492,"[1995.05, 2025.05]",5.255142,7.956656,12.046939,27.616519
1493,"[1995.06, 2025.06]",5.250041,7.947003,12.029402,27.562931
1494,"[1995.07, nan]",NaN,NaN,NaN,NaN


In [ ]:

boxplot = boxplot_returns(returns,portfolio_columns=)
boxplot
